In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import json
import pickle

## 데이터 불러오기
- 강남구 전체 음식점 데이터

In [2]:
merged = pd.read_csv("data/merged.csv")

In [3]:
all_df = pd.read_csv("data/all_cafeteria_mobeom.csv")
all_df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울특별시 강남구 도산대로55길 20 (청담동),배초향,한식,203520.881091,446950.285626,NaN,NaN,청담동,NaN
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,"서울특별시 강남구 봉은사로 524, 1층 (삼성동, 인터컨티넨탈 서울 코엑스)",브래서리,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,모범식당지정
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,"서울특별시 강남구 봉은사로 524, 1층 (삼성동, 인터컨티넨탈 서울 코엑스)",로비라운지,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN


In [4]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12528 entries, 0 to 12527
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        12528 non-null  int64  
 1   MGTNO             12528 non-null  object 
 2   APVPERMYMD        12528 non-null  object 
 3   TRDSTATENM        12528 non-null  object 
 4   SITEAREA          12513 non-null  object 
 5   SITEPOSTNO        12518 non-null  object 
 6   SITEWHLADDR       12518 non-null  object 
 7   RDNWHLADDR        12472 non-null  object 
 8   BPLCNM            12528 non-null  object 
 9   UPTAENM           12527 non-null  object 
 10  X                 12475 non-null  float64
 11  Y                 12475 non-null  float64
 12  TRDPJUBNSENM      2237 non-null   object 
 13  WTRSPLYFACILSENM  2976 non-null   object 
 14  dong              12528 non-null  object 
 15  exemplary_diner   316 non-null    object 
dtypes: float64(2), int64(1), object(13)
memo

In [5]:
# 강남구 전체 데이터 식당 이름 결측치 확인 -> 없음
all_df["BPLCNM"].isnull().sum()

0

## request 적용

### 강남구 모범음식점 지정 데이터셋과 이름 비교
- ex. 삼성물산(주)10꼬르소꼬모서울카페, 솥내음 스타필드 코엑스몰점 -> 모범음식점 지정, 전체 데이터셋 둘다 모두 동일한 이름으로 지정

In [6]:
all_df[all_df.BPLCNM.str.contains("꼬르소꼬모")]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner
3020,3020,3220000-101-2008-00093,2008-02-20,영업/정상,350.00,135-954,서울특별시 강남구 청담동 79-0 지하1층,"서울특별시 강남구 압구정로 416 (청담동,지하1층)",삼성물산(주)10꼬르소꼬모서울카페,경양식,203679.945734,447187.003951,NaN,NaN,청담동,모범식당지정


In [7]:
all_df[all_df.BPLCNM.str.contains("솥내음")]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner
8374,8374,3220000-101-2020-00643,2020-06-16,영업/정상,74.56,135-731,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 지하1층 O-107호 (삼성동)",솥내음 스타필드 코엑스몰점,한식,205130.591679,445590.096838,NaN,NaN,삼성동,모범식당지정


### 가게 이름 전처리

In [8]:
# 가게 이름 전처리
# "(주)" -> 적혀 있는 가게 이름에서 "(주)" 제거
def find_string(txt):
    if '주)' in txt:
        target_index = txt.index('주)')
        if target_index > len(txt) // 2:
            return txt[:txt.index('주)') - 1]
        else:
            return txt[txt.index('주)') + 2:]
    else:
        return txt

# find_string 함수 적용 + "~점"으로 끝나는 가게 이름에서 "점" 제거
all_df["keyword"] = all_df['BPLCNM'].apply(find_string).str.split('(').str[0].str.rstrip('점')

In [9]:
# # 업소 이름 뒤에 위치한 동 이름 붙임
# def plus_dong(txt):
#     if txt["dong"][:-1] not in txt["keyword"]:
#         txt["keyword"] = txt["keyword"] + " " + txt["dong"][:-1]
#     else:
#         txt["keyword"] = txt["keyword"]
#     return txt

In [10]:
# plus_dong 함수 적용
# all_df = all_df.apply(plus_dong, axis=1)

In [11]:
all_df["keyword"].head(3)

0      배초향
1     브래서리
2    로비라운지
Name: keyword, dtype: object

### 지번 주소 전처리

In [12]:
all_df[all_df.BPLCNM.str.contains("아야진생태찌개")]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword
2924,2924,3220000-101-2007-00781,20070928,영업/정상,185.42,135878,서울특별시 강남구 삼성동 150-2 (101호),"서울특별시 강남구 삼성로108길 8 (삼성동,(101호))",아야진생태찌개,한식,204771.670253,445508.924495,NaN,NaN,삼성동,모범식당지정,아야진생태찌개
4645,4645,3220000-101-2013-00416,20130531,영업/정상,253.28,135930,"서울특별시 강남구 역삼동 797-11번지 지상1,2,3,층","서울특별시 강남구 논현로 329, 지상1,2,3층 (역삼동)",아야진생태찌개 역삼점,한식,203418.921771,443599.760104,NaN,NaN,역삼동,모범식당지정,아야진생태찌개 역삼


In [13]:
all_df[all_df.BPLCNM.str.contains("아야진생태찌개")]["SITEWHLADDR"].values[0].split(" ")

['서울특별시', '강남구', '삼성동', '150-2', '(101호)']

In [29]:
# def generate_address(txt):
#     try:
#         if txt.split(' ')[5]:
#             if txt.split(' ')[5][-1] == '호':
#                 return txt.split(' ')[2] + ' ' + txt.split(' ')[4].rstrip('번지') + '-' + txt.split(' ')[5].rstrip('호')
#             else:
#                 return txt.split(' ')[2] + ' ' + txt.split(' ')[4].rstrip('번지')
#         else:
#             return txt.split(' ')[2] + ' ' + txt.split(' ')[4].rstrip('번지')
            
#     except:
#         return False

### 도로명 주소 전처리
- 지번 주소명에서 규칙을 찾기 힘들어 도로명 주소로 대체
- 도로명 주소 결측치 존재 (56개)
    - `.fillna("")`로 빈칸으로 일단 채우고 나서 전처리 진행

In [14]:
all_df["RDNWHLADDR"] = all_df["RDNWHLADDR"].fillna("")

In [15]:
def road_split(txt):
    txt = txt.split(",")[0].replace("서울특별시", "서울")
    txt = txt.split("(")[0]
    return txt

In [16]:
all_df["RDNWHLADDR"] = all_df["RDNWHLADDR"].apply(road_split)

In [17]:
key = '923dc9604178f2a7aa0979ae38029a34'
kakao_result = []

keywords = all_df['keyword'].tolist()
no_model_keyword = all_df[all_df.exemplary_diner.isna()]["keyword"].tolist()

### 카카오맵 상세보기 id 불러오기

In [18]:
print(len(keywords))
print(len(no_model_keyword))
print("모범음식점 개수 :", len(keywords)-len(no_model_keyword), "개")

12528
12212
모범음식점 개수 : 316 개


In [22]:
kakao_result = []
rows = []
ids = []
# ids2 = []
# ids3 = []

for idx, keyword in enumerate(keywords):
# for idx, keyword in enumerate(keywords[10669:]):

    if keyword == np.nan:
        ids.append("")
        continue


    url = 'https://dapi.kakao.com/v2/local/search/keyword.json' 
    params = {'query': keyword, 'page': 1}
    headers = {"Authorization": f'''KakaoAK {key}'''}
    response = requests.get(url, params=params, headers=headers)
    this_result = []
    this_id = []
    try:
        for row in json.loads(response.content)['documents']:
            rows.append(row)
            cond_gangnam = row['address_name'].split(' ')[0] == '서울' and row['address_name'].split(' ')[1] == '강남구'
            # cond_addr = generate_address(df.loc[idx, 'SITE_ADDR']) == row['address_name'][7:]
            # cond_road_addr = df.loc[idx, 'SITE_ADDR_RD'].replace('특별시', '').split(',')[0] == row['road_address_name']
            cond_road_addr = all_df.loc[idx, 'RDNWHLADDR'][:-1] in row['road_address_name']

            # if cond_gangnam and (cond_addr or cond_road_addr):
            if cond_gangnam and cond_road_addr:
                this_result.append({
                    '검색어': keyword,
                    '상호명': row['place_name'],
                    '지번주소': row['address_name'],
                    '도로명주소': row['road_address_name'],
                    'id': row['id']
                })
                this_id.append(row['id'])

        # ids.append(this_id if this_id else None)
        # ids2.append(this_id if this_id else None)
        ids.append(this_id if this_id else None)

    except:
        pass

In [23]:
response.text

'{"documents":[],"meta":{"is_end":true,"pageable_count":0,"same_name":{"keyword":"김가네 ","region":[],"selected_region":"서울 강남구 자곡동"},"total_count":0}}'

In [24]:
ids

[None,
 ['10825881'],
 ['727661240'],
 ['21550576'],
 ['17157328', '26597965'],
 None,
 ['16055100'],
 ['26983298'],
 None,
 ['16036307'],
 ['9614833'],
 ['1750491579'],
 None,
 None,
 None,
 None,
 None,
 ['19351143'],
 ['8299376'],
 None,
 ['10360096'],
 None,
 None,
 ['16499758'],
 None,
 ['14982154'],
 None,
 ['16677411'],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['7867924'],
 ['24178025'],
 ['1141385651'],
 ['1552651008'],
 None,
 ['11057157'],
 None,
 ['21222844'],
 None,
 ['1042080004'],
 ['1660939960'],
 None,
 None,
 ['16556376'],
 ['7889948'],
 ['25612762'],
 ['285392408'],
 ['10252806', '1905645499'],
 ['8348458'],
 ['2097387035'],
 ['607030045'],
 ['1741104454'],
 None,
 None,
 None,
 ['27344977'],
 ['21237816'],
 ['8110399'],
 None,
 ['8055932'],
 None,
 ['16054592'],
 ['14603291'],
 None,
 ['1869730586'],
 None,
 None,
 ['18738555'],
 ['8156521', '26646332'],
 None,
 None,
 ['10216264'],
 ['13057957'],
 ['16036952'],
 ['8410723'],
 None,
 None,
 None,
 None,
 ['1

In [84]:
# 10669 ~ 10793 index 까지 id 다시 한번 찾음
ids2

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['1053416793'],
 None,
 None,
 None,
 ['646221301', '2062558044', '549441286', '1759469103', '169864988'],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['632636561', '923738207'],
 None,
 None,
 ['1966152470'],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['231243793',
  '1413212886',
  '1561037207',
  '1323949073',
  '1014051301',
  '721149197',
  '805389038',
  '223745869',
  '1819478245'],
 None,
 None,
 None,
 None,
 ['16998823'],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 Non

In [74]:
len(ids)

10669

In [85]:
len(ids2)

124

In [119]:
10669 + 124

10793

In [25]:
# id 컬럼 생성
all_df["id"] = pd.Series(ids)

In [129]:
# id 컬럼의 10669 인덱스부터 10792까지 다시 찾은 값 (ids2)으로 대체
all_df["id"].loc[10669:10792] = pd.Series(ids2, index=range(10669, 10793))

C:\Users\aryij\AppData\Local\Temp\ipykernel_30148\657290561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df["id"].loc[10669:10792] = pd.Series(ids2, index=range(10669, 10793))


In [136]:
all_df[all_df.exemplary_diner.notnull()].isnull().sum()

Unnamed: 0            0
MGTNO                 0
APVPERMYMD            0
TRDSTATENM            0
SITEAREA              0
SITEPOSTNO            0
SITEWHLADDR           0
RDNWHLADDR            0
BPLCNM                0
UPTAENM               0
X                     1
Y                     1
TRDPJUBNSENM        205
WTRSPLYFACILSENM    177
dong                  0
exemplary_diner       0
keyword               0
id                  140
id_mod                0
dtype: int64

In [137]:
all_df[all_df.exemplary_diner.isna()].isnull().sum()

Unnamed: 0              0
MGTNO                   0
APVPERMYMD              0
TRDSTATENM              0
SITEAREA               15
SITEPOSTNO             10
SITEWHLADDR            10
RDNWHLADDR              0
BPLCNM                  0
UPTAENM                 1
X                      52
Y                      52
TRDPJUBNSENM        10086
WTRSPLYFACILSENM     9375
dong                    0
exemplary_diner     12212
keyword                 0
id                   6093
id_mod                  0
dtype: int64

## id 컬럼 결측치

In [26]:
# request로 가져온 id 컬럼의 결측치 빈칸으로 대체
all_df["id_mod"] = all_df["id"].fillna(" ")

In [27]:
# id 컬럼이 list로 되어 있어, 문자열로 변경
all_df["id_mod"] = all_df["id_mod"].apply(lambda x: x[0] if x is not None else x)

In [174]:
all_df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,203520.881091,446950.285626,NaN,NaN,청담동,NaN,배초향,None,
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,모범식당지정,브래서리,[10825881],10825881
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비라운지,[727661240],727661240


## merged 데이터와 병합

In [149]:
merged.head(3)

,CGG_CODE,ASGN_YY,ASGN_SNO,APPL_YMD,ASGN_YMD,UPSO_NM,SITE_ADDR_RD,SITE_ADDR,PERM_NT_NO,SNT_UPTAE_NM,MAIN_EDF,TRDP_AREA,ADMDNG_NM,GRADE_FACIL_GBN,UPSO_SITE_TELNO,UPSO_NM_MOD,UPSO_NM_LONG,SITE_ADDR_RD_SPLIT,name,url
0,3220000,2014,160,20141006,20141006,이도곰탕,"서울특별시 강남구 논현로94길 29-5, 지상1층,지상2층 (역삼동)",서울특별시 강남구 역삼동 671번지 17호,3220000-101-2001-24197,한식,곰탕,194.41,역삼1동,상수도전용,02 5010738,이도곰탕,이도곰탕 역삼1동,서울 강남구 논현로94길 29-5,이도곰탕 역삼1동,https://place.map.kakao.com/14569639
1,3220000,2021,139,20181001,20211116,솥내음 스타필드 코엑스몰점,"서울특별시 강남구 영동대로 513, 코엑스 지하1층 O-107호 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,3220000-101-2020-00643,한식,직화불백,74.56,삼성1동,NaN,NaN,솥내음 스타필드 코엑스몰,솥내음 스타필드 코엑스몰 삼성1동,서울 강남구 영동대로 513,솥내음 스타필드 코엑스몰 삼성1동,https://place.map.kakao.com/710342577
2,3220000,2010,24,20100210,20100210,스시히로바,"서울특별시 강남구 삼성로 620, 블래스톤리조트 지상1층 (삼성동)",서울특별시 강남구 삼성동 70번지 블래스톤리조트,3220000-101-2002-00383,일식,초밥,216.56,삼성1동,상수도전용,02 5155511,스시히로바,스시히로바 삼성1동,서울 강남구 삼성로 620,스시히로바 삼성1동,https://place.map.kakao.com/7974994


In [28]:
# merged 데이터프레임의 url 결측치 빈칸으로 대체
merged["url"] = merged["url"].fillna(" ")

# url 맨 뒤 숫자만 가져온다
merged["url_split"] = merged["url"].apply(lambda x: x.split("/")[-1])

In [29]:
# 업소 이름 맨 뒤에 있는 동네 이름 제거
merged["upso_split"] = merged["UPSO_NM_LONG"].apply(lambda x: " ".join(x.split()[:-1]))
merged["upso_split"].head(3)

0             이도곰탕
1    솥내음 스타필드 코엑스몰
2            스시히로바
Name: upso_split, dtype: object

In [30]:
# merged 데이터프레임의 업소 이름, url을 딕셔너리 형태로 생성
dict_merged_upso_url = merged[["upso_split", "url_split"]].set_index("upso_split").to_dict()["url_split"]
dict_merged_upso_url

{'이도곰탕': '14569639',
 '솥내음 스타필드 코엑스몰': '710342577',
 '스시히로바': '7974994',
 '서백자 간장게장': '24663994',
 '봉산집': '26317333',
 '더드림': '27335171',
 '국민한우집 삼성역': '1259415579',
 '이즈미': '21414358',
 '홍영재 장수 청국장': '19838635',
 '등대': '8958910',
 '': '12543930',
 '김태준의 탕탕집': '170461079',
 '불이아': '10452008',
 '경성면옥': ' ',
 '밀밭': '26896916',
 '안동찜닭 해물떡찜': '1211150938',
 '니뽕내뽕(코엑스': '27441566',
 '세화정육': '631696368',
 '신의주 부대찌개': '20642173',
 '지호한방삼계탕': '19589343',
 '차이나타운': '10769361',
 '오발탄에프에스': '1456391705',
 '가원': '1392972465',
 '대흥정 엘에이 갈비': '20007819',
 '아야진생태찌개': '19974580',
 '수림복국': '17154422',
 '평가옥': '8117481',
 '바오바오': '26353603',
 '곰바위': '10157717',
 '황소곱창': '333031201',
 '삼성동': '27274125',
 '프로간장게장': '15227243',
 '해랑': '26777815',
 '본가 신의주 찹쌀순대': '8440324',
 '이조식당': '1544581685',
 '공수간 코엑스몰': '26894267',
 '명동칼국수': '16054618',
 '도모다찌': '13308284',
 '예가낙지마을': '1751509695',
 '동경': '16986529',
 '토말': '11594198',
 '엠브이샤브샤브': '11108322',
 '조수사': '10800261',
 '토담골': '8107787',
 '시골진지상': '9186290',

In [179]:
dict_merged_upso_url.keys()

dict_keys(['이도곰탕', '솥내음 스타필드 코엑스몰', '스시히로바', '서백자 간장게장', '봉산집', '더드림', '국민한우집 삼성역', '이즈미', '홍영재 장수 청국장', '등대', '', '김태준의 탕탕집', '불이아', '경성면옥', '밀밭', '안동찜닭 해물떡찜', '니뽕내뽕(코엑스', '세화정육', '신의주 부대찌개', '지호한방삼계탕', '차이나타운', '오발탄에프에스', '가원', '대흥정 엘에이 갈비', '아야진생태찌개', '수림복국', '평가옥', '바오바오', '곰바위', '황소곱창', '삼성동', '프로간장게장', '해랑', '본가 신의주 찹쌀순대', '이조식당', '공수간 코엑스몰', '명동칼국수', '도모다찌', '예가낙지마을', '동경', '토말', '엠브이샤브샤브', '조수사', '토담골', '시골진지상', '가득드림', '미로정', '한촌설렁탕 선릉역', '유장원보쌈', '강남어시장', '원조닭칼국수', '국고집', '김수사', '박대박', '무탄', '전주명가', '계승집', '명동칼국수샤브샤브', '천미미', '명가한식', '바다를 훔친 언니', '아오끼', '송화', '하동관', '두릉도원 북촌순두부', '토담골49번지', '연화산', '천하일품추어탕', '내림손 삼계탕', '아구본가 첨벙 신사본', '명동관', '우밀가 상록', '무화잠', '금강수림', '진수사', '햇무리', '옛날집', '두껍삼', '케르반레스토랑 코엑스', '놀부보쌈과 부대찌게 삼성1호', '스카이라운지', '브래서리', '하꼬네', '아시안라이브', '새마을식당', '영이네', '구이가', '영암집', '만두의 전설', '언양불고기', '리브팜 압구정', '골목길', '초심한우 정육식당', '노다지 삼겹살', '사조참치삼성', '호보식당', '보슬보슬 압구정본', '남도랑', '논골집', '해남집', '윤화돈까스 기사식당', '카페비스타', '와인앤다인', '대운정', '남도사계고운님', '주식회사 원강', '무등산', '오레노카츠',

In [178]:
dict_merged_upso_url.values()

dict_values(['14569639', '710342577', '7974994', '24663994', '26317333', '27335171', '1259415579', '21414358', '19838635', '8958910', '12543930', '170461079', '10452008', ' ', '26896916', '1211150938', '27441566', '631696368', '20642173', '19589343', '10769361', '1456391705', '1392972465', '20007819', '19974580', '17154422', '8117481', '26353603', '10157717', '333031201', '27274125', '15227243', '26777815', '8440324', '1544581685', '26894267', '16054618', '13308284', '1751509695', '16986529', '11594198', '11108322', '10800261', '8107787', '9186290', '24808019', '1897959308', '18279510', '8543163', '1937669323', '9956715', '1292909425', '10201006', '1377334197', '1696571508', ' ', '265003060', '12572463', '2010643844', '1186348635', '372391352', '16679184', '160405096', '26428654', ' ', '8127657', '295975536', '1325964549', '403548943', ' ', '19852302', '1928763865', '683635436', '14659473', '8095238', '1580031159', '17953166', '2011092566', ' ', '2025057104', '8106111', '10825881', '14

In [31]:
# merged 데이터프레임의 업소 이름, url을 딕셔너리 형태로 생성한 것에서 all_df의 업소 이름에 맞는 id (숫자)를 가져오는 함수 생성
def replace_url(x):
    name = x["keyword"]
    if name in dict_merged_upso_url.keys():
        x["id_mod2"] = dict_merged_upso_url[name]
    else:
        x["id_mod2"] = x["id_mod"]
    return x

In [32]:
# 함수 적용
all_df_mod = all_df.apply(replace_url, axis=1)
all_df_mod.head()

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,203520.881091,446950.285626,NaN,NaN,청담동,NaN,배초향,None,,
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,모범식당지정,브래서리,[10825881],10825881,10825881
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비라운지,[727661240],727661240,727661240
3,3,3220000-101-1899-20463,19991006,영업/정상,282.59,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비바,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비바,[21550576],21550576,21550576
4,4,3220000-101-1899-20500,19991012,영업/정상,"1,157.90",135873,서울특별시 강남구 삼성동 109-6,서울 강남구 봉은사로 637,노블발렌티,한식,205662.873886,445913.719788,결혼예식장주변,상수도전용,삼성동,NaN,노블발렌티,"[17157328, 26597965]",17157328,17157328


In [33]:
# merged 데이터프레임에서 url 빈 것들 개수
merged[merged.url == " "].shape

(9, 22)

In [34]:
# replace_url 함수 적용한 all_df_mod 데이터프레임에서 url 빈 것들 개수
# -> 9개여야 하는데 23개이다
all_df_mod[(all_df_mod.exemplary_diner.notnull()) & (all_df_mod.id_mod2 == " ")].shape

(23, 20)

In [35]:
# replace_url 함수 적용한 all_df_mod 데이터프레임에서 url 빈 것들 업소 이름 확인
all_df_mod[(all_df_mod.exemplary_diner.notnull()) & (all_df_mod.id_mod2 == " ")]["keyword"]

235      아구본가 첨벙 신사본
430               나무
695      신의주 부대찌개 삼성
702             전주명가
754       지강한식당 압구정본
1183        원조본점닭한마리
1691        삼성동 명인등심
1702        뉴욕바닷가재청담
1769            경성면옥
2785             단스시
3378        아구본가첨벙대치
3789         베이징코야강남
4980       별난횟집 역삼직영
5126       고궁수라간 코엑스
5134      두릉도원 북촌순두부
5208             멘무샤
5224            니뽕내뽕
5225     케르반레스토랑 코엑스
5695              양문
6124        두껍삼 역삼직영
6272             대운정
6579         분짜라붐 역삼
10996        수백당 삼성동
Name: keyword, dtype: object

In [36]:
# replace_url 함수 적용한 all_df_mod 데이터프레임에서 url 빈 것들 검색
# -> url 숫자 직접 채워 넣음
no_urls = [np.nan,
           "15500067",
           "20642173",
           np.nan,
           "2030032655",
           "9956715",
           "27274125",
           "8216639",
           np.nan,
           "8306074", # 단스시
           "16534533",
           np.nan,
           "1891715448",
           np.nan,
           np.nan,
           np.nan,
           "27441566",
           np.nan,
           "1054657548",
           "2011092566",
           np.nan,
           "1811506643",
           "213776157"]

In [37]:
no_urls_filled = pd.Series(no_urls, index=all_df_mod[(all_df_mod.exemplary_diner.notnull()) & (all_df_mod.id_mod2 == " ")]
["id_mod2"].index)

In [341]:
no_urls_filled.index

Index([  235,   430,   695,   702,   754,  1183,  1691,  1702,  1769,  2785,
        3378,  3789,  4980,  5126,  5134,  5208,  5224,  5225,  5695,  6124,
        6272,  6579, 10996],
      dtype='int64')

In [342]:
no_urls_filled.values

array([nan, '15500067', '20642173', nan, '2030032655', '9956715',
       '27274125', '8216639', nan, '8306074', '16534533', nan,
       '1891715448', nan, nan, nan, '27441566', nan, '1054657548',
       '2011092566', nan, '1811506643', '213776157'], dtype=object)

In [38]:
# id_mod2 (url) 빈 숫자 채워 넣는다
all_df_mod.loc[no_urls_filled.index, "id_mod2"] = no_urls_filled.values

In [39]:
all_df_mod[(all_df_mod.exemplary_diner.notnull()) & (all_df_mod.id_mod2.isna())].shape

(9, 20)

In [40]:
all_df_mod[all_df_mod.keyword.isna()]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2


In [354]:
# keyword가 비어있는 row의 id_mod2 컬럼이 12543930로 들어가 있는 것 확인
all_df_mod[all_df_mod.keyword == ""].shape

(11, 20)

In [42]:
all_df_mod[all_df_mod.id_mod2=="12543930"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
247,247,3220000-101-1990-07507,19901101,영업/정상,31.73,135860,서울특별시 강남구 도곡동 954-19번지,서울 강남구 남부순환로355길 13,(신)오양절면,한식,203087.887972,442628.722914,기타,상수도전용,도곡동,NaN,,[338342608],338342608,12543930
994,994,3220000-101-1998-12691,19981029,영업/정상,94.63,135515,서울특별시 강남구 역삼동 787-1번지,서울 강남구 논현로64길 22,(중화요리)남경,중국식,203668.426416,443485.604196,기타,상수도전용,역삼동,NaN,,[10960059],10960059,12543930
1432,1432,3220000-101-2000-22499,2000-09-27,영업/정상,677.61,135-731,서울특별시 강남구 삼성동 159 코엑스,서울 강남구 영동대로 513,(유)아웃백스테이크하우스코리아 코엑스스타필드점,경양식,205130.591679,445590.096838,NaN,NaN,삼성동,NaN,,[16877892],16877892,12543930
3543,3543,3220000-101-2010-00073,20100204,영업/정상,28.00,135824,서울특별시 강남구 논현동 157-0번지 지상1층,서울 강남구 강남대로122길 52,스파게티 스토리,김밥(도시락),202334.481225,445197.044292,NaN,상수도전용,논현동,NaN,스파게티 스토리,[12543930],12543930,12543930
3866,3866,3220000-101-2011-00114,20110223,영업/정상,81.78,135955,서울특별시 강남구 청담동 122-7 1층,서울 강남구 도산대로85길 14,(첼시스 하이볼) Chelsea's HIGHBALL,일식,204279.144297,447023.007955,NaN,NaN,청담동,NaN,,[19153490],19153490,12543930
4031,4031,3220000-101-2011-00686,20110811,영업/정상,49.50,135515,서울특별시 강남구 역삼동 781-41 지상1층102호,서울 강남구 논현로66길 9,(SOS)커피 역삼점,경양식,203566.263883,443554.896146,NaN,NaN,역삼동,NaN,,[14600986],14600986,12543930
4511,4511,3220000-101-2013-00033,20130117,영업/정상,177.28,135911,서울특별시 강남구 역삼동 646-4번지,서울 강남구 강남대로94길 86,(이층) 맥주창고,경양식,202951.131852,444272.491202,NaN,NaN,역삼동,NaN,,None,,12543930
6054,6054,3220000-101-2016-00899,2016-10-21,영업/정상,507.39,135-825,서울특별시 강남구 논현동 165,서울 강남구 강남대로 502,(유)아웃백스테이크하우스코리아 강남점,경양식,202024.615630,444994.659485,NaN,NaN,논현동,NaN,,[522500805],522500805,12543930
8697,8697,3220000-101-2020-01279,20201030,영업/정상,268.98,135896,서울특별시 강남구 신사동 637-11,서울 강남구 압구정로42길 12,(끌레오)CLO,한식,203049.750409,447323.452649,NaN,NaN,신사동,NaN,,[1495236287],1495236287,12543930
9148,9148,3220000-101-2021-00506,20210503,영업/정상,33.00,135949,서울특별시 강남구 청담동 9-12,서울 강남구 학동로77길 50,(유노시엘) EUNOSIERE,기타,204171.424581,446636.433530,NaN,NaN,청담동,NaN,,[118575691],118575691,12543930


In [44]:
all_df_mod.loc[247:250]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
247,247,3220000-101-1990-07507,19901101,영업/정상,31.73,135860,서울특별시 강남구 도곡동 954-19번지,서울 강남구 남부순환로355길 13,(신)오양절면,한식,203087.887972,442628.722914,기타,상수도전용,도곡동,NaN,,[338342608],338342608,12543930
248,248,3220000-101-1990-07519,1990-06-22,영업/정상,97.90,135-841,서울특별시 강남구 대치동 906,서울 강남구 선릉로76길 24,양문,한식,204574.569178,444393.323912,NaN,NaN,대치동,NaN,양문,[912331600],912331600,912331600
249,249,3220000-101-1990-07548,19900330,영업/정상,210.71,135877,서울특별시 강남구 삼성동 143-30번지,서울 강남구 삼성로91길 28,강성환차이나,중국식,204698.725555,444950.288061,유흥업소밀집지역,상수도전용,삼성동,NaN,강성환차이나,None,,
250,250,3220000-101-1990-07591,19900817,영업/정상,28.89,135928,서울특별시 강남구 역삼동 773-8번지,서울 강남구 언주로 323,계몽돈까스,분식,203915.714136,443839.427549,기타,상수도전용,역삼동,NaN,계몽돈까스,[1411235044],1411235044,1411235044


In [48]:
all_df_mod.shape[0]

12528

In [51]:
all_df_mod.loc[12500:12527]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
12500,12500,3220000-101-2024-00487,2024-04-18,영업/정상,13.90,135-897,서울특별시 강남구 신사동 657-26,서울 강남구 도산대로51길 37,쭈니네 츄러스 압구정점,기타,203290.131851,447051.960056,NaN,NaN,신사동,NaN,쭈니네 츄러스 압구정,None,,
12501,12501,3220000-101-2024-00488,2024-04-19,영업/정상,30.00,135-916,서울특별시 강남구 역삼동 683-11 한국빌딩,서울 강남구 선릉로 565,주식회사 컬처앤비,기타,203830.254782,445195.243114,NaN,NaN,역삼동,NaN,주식회사 컬처앤비,None,,
12502,12502,3220000-101-2024-00489,2024-04-19,영업/정상,59.79,135-935,서울특별시 강남구 역삼동 826-14 강남358타워,서울 강남구 강남대로 358,모닥스낵 강남358타워점,분식,202557.007831,443648.622540,NaN,NaN,역삼동,NaN,모닥스낵 강남358타워,None,,
12503,12503,3220000-101-2024-00490,2024-04-19,영업/정상,15.00,135-833,서울특별시 강남구 논현동 278-14,서울 강남구 봉은사로51길 3-3,구름족,한식,203686.143918,445344.679446,NaN,NaN,논현동,NaN,구름족,None,,
12504,12504,3220000-101-2024-00491,2024-04-19,영업/정상,46.45,135-501,서울특별시 강남구 대치동 989-2,서울 강남구 도곡로 510,싸움의 고수 대치점,한식,205408.737773,444079.844884,NaN,NaN,대치동,NaN,싸움의 고수 대치,None,,
12505,12505,3220000-101-2024-00492,2024-04-19,영업/정상,132.00,135-917,서울특별시 강남구 역삼동 696-17,서울 강남구 선릉로93길 10,남부 선릉점,식육(숯불구이),204132.541000,444816.366138,NaN,NaN,역삼동,NaN,남부 선릉,None,,
12506,12506,3220000-101-2024-00493,2024-04-19,영업/정상,50.04,135-502,서울특별시 강남구 대치동 997 오로라빌딩,서울 강남구 테헤란로 624,로스터리 젠,기타,205758.453961,445233.652528,NaN,NaN,대치동,NaN,로스터리 젠,None,,
12507,12507,3220000-101-2024-00494,2024-04-19,영업/정상,81.42,135-925,서울특별시 강남구 역삼동 752-38,서울 강남구 역삼로11길 3-3,케이 바(K Bar),감성주점,202941.543753,443615.146844,NaN,NaN,역삼동,NaN,케이 바,None,,
12508,12508,3220000-101-2024-00495,2024-04-19,영업/정상,391.53,135-931,서울특별시 강남구 역삼동 816-6,서울 강남구 테헤란로1길 29,팀호완 강남점,중국식,202344.864503,444215.048140,NaN,NaN,역삼동,NaN,팀호완 강남,None,,
12509,12509,3220000-101-2024-00496,2024-04-22,영업/정상,0.00,135-902,서울특별시 강남구 압구정동 494 갤러리아백화점,서울 강남구 압구정로 343,본작(한시적),기타,203470.848439,447369.579852,NaN,NaN,압구정동,NaN,본작,None,,


In [41]:
all_df_mod[all_df_mod.id_mod2=="12543930"].shape

(12, 20)

In [363]:
# id_mod2에 12543930로 중복되는 값들 id_mod에서 index를 찾아 대체
dup_index = all_df_mod[all_df_mod.keyword == ""].index
all_df_mod.loc[dup_index, "id_mod2"] = all_df_mod["id_mod"].loc[dup_index]

In [364]:
all_df_mod[all_df_mod.keyword == ""]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
247,247,3220000-101-1990-07507,19901101,영업/정상,31.73,135860,서울특별시 강남구 도곡동 954-19번지,서울 강남구 남부순환로355길 13,(신)오양절면,한식,203087.887972,442628.722914,기타,상수도전용,도곡동,NaN,,[338342608],338342608,338342608
994,994,3220000-101-1998-12691,19981029,영업/정상,94.63,135515,서울특별시 강남구 역삼동 787-1번지,서울 강남구 논현로64길 22,(중화요리)남경,중국식,203668.426416,443485.604196,기타,상수도전용,역삼동,NaN,,[10960059],10960059,10960059
1432,1432,3220000-101-2000-22499,2000-09-27,영업/정상,677.61,135-731,서울특별시 강남구 삼성동 159 코엑스,서울 강남구 영동대로 513,(유)아웃백스테이크하우스코리아 코엑스스타필드점,경양식,205130.591679,445590.096838,NaN,NaN,삼성동,NaN,,[16877892],16877892,16877892
3866,3866,3220000-101-2011-00114,20110223,영업/정상,81.78,135955,서울특별시 강남구 청담동 122-7 1층,서울 강남구 도산대로85길 14,(첼시스 하이볼) Chelsea's HIGHBALL,일식,204279.144297,447023.007955,NaN,NaN,청담동,NaN,,[19153490],19153490,19153490
4031,4031,3220000-101-2011-00686,20110811,영업/정상,49.50,135515,서울특별시 강남구 역삼동 781-41 지상1층102호,서울 강남구 논현로66길 9,(SOS)커피 역삼점,경양식,203566.263883,443554.896146,NaN,NaN,역삼동,NaN,,[14600986],14600986,14600986
4511,4511,3220000-101-2013-00033,20130117,영업/정상,177.28,135911,서울특별시 강남구 역삼동 646-4번지,서울 강남구 강남대로94길 86,(이층) 맥주창고,경양식,202951.131852,444272.491202,NaN,NaN,역삼동,NaN,,None,,
6054,6054,3220000-101-2016-00899,2016-10-21,영업/정상,507.39,135-825,서울특별시 강남구 논현동 165,서울 강남구 강남대로 502,(유)아웃백스테이크하우스코리아 강남점,경양식,202024.615630,444994.659485,NaN,NaN,논현동,NaN,,[522500805],522500805,522500805
8697,8697,3220000-101-2020-01279,20201030,영업/정상,268.98,135896,서울특별시 강남구 신사동 637-11,서울 강남구 압구정로42길 12,(끌레오)CLO,한식,203049.750409,447323.452649,NaN,NaN,신사동,NaN,,[1495236287],1495236287,1495236287
9148,9148,3220000-101-2021-00506,20210503,영업/정상,33.00,135949,서울특별시 강남구 청담동 9-12,서울 강남구 학동로77길 50,(유노시엘) EUNOSIERE,기타,204171.424581,446636.433530,NaN,NaN,청담동,NaN,,[118575691],118575691,118575691
9354,9354,3220000-101-2021-00858,20210708,영업/정상,159.25,135915,서울특별시 강남구 역삼동 676 센터필드,서울 강남구 테헤란로 231,(유)한국집 센터필드 역삼점,한식,203599.062772,444556.062267,NaN,NaN,역삼동,NaN,,[1872560550],1872560550,1872560550


In [366]:
# url 채운 파일 csv로 저장
all_df_mod.to_csv("data/request_reviews.csv", index=False, encoding="utf-8")